In [1]:
from flytekit.configuration import platform
def print_console_url(exc):
    print("http://{}/console/projects/{}/domains/{}/executions/{}".format(platform.URL.get(), exc.id.project, exc.id.domain, exc.id.name))

# REPL-sorta

### 1) Run locally
Flyte allows you to run the exact tasks that run remotely locally to iterate, unit test, and fix bugs.

In [7]:
from workflows.recipe_7.interactive_python import interactive_python

r = interactive_python.unit_test(a=2.0, b=3.0)
print('2+3={}'.format(r['out']))

r = interactive_python.unit_test(a=-5.0, b=3.0)
print('-5+3={}'.format(r['out']))


2+3=5.0



-5+3=-2.0


### 2) Build a docker image
To ensure repeatablity, we recommend building a docker image with a new version to reflect any code change.

Note: You need to have push permissions on a docker hub repo (or any other docker registry) to execute the docker push command.

In [6]:
from os import environ
environ["version"]="24"
!docker build -t docker.io/lyft/flytesnacks:datacouncil-$version .
!docker push docker.io/lyft/flytesnacks:datacouncil-$version

Sending build context to Docker daemon  284.2kB
Step 1/19 : FROM ubuntu:bionic
 ---> 775349758637
Step 2/19 : WORKDIR /root
 ---> Using cache
 ---> 9e71f2c2950c
Step 3/19 : ENV VENV /opt/venv
 ---> Using cache
 ---> 7605b00f67e3
Step 4/19 : ENV LANG C.UTF-8
 ---> Using cache
 ---> 197d97963594
Step 5/19 : ENV LC_ALL C.UTF-8
 ---> Using cache
 ---> f424c74f8294
Step 6/19 : ENV PYTHONPATH /root
 ---> Using cache
 ---> ce39083a6d7c
Step 7/19 : RUN :     && apt-get update && apt-get install -y python3.6 python3.6-venv python3.6-dev make build-essential libssl-dev python3-venv python3-pip curl
 ---> Using cache
 ---> fbbef8af0eb2
Step 8/19 : RUN apt-get update && apt-get install -y libsm6 libxext6 libxrender-dev
 ---> Using cache
 ---> 61cfd96195b8
Step 9/19 : RUN pip3 install awscli
 ---> Using cache
 ---> 9ec5fd79ae76
Step 10/19 : RUN python3.6 -m venv ${VENV}
 ---> Using cache
 ---> c9f5e3d2048a
Step 11/19 : RUN ${VENV}/bin/pip install wheel
 ---> Using cache
 ---> 14c2e932b36c
Step 12/1

### 2) Register a notebook task
FlyteKit can discover all tasks, workflows and launchplans in a given python package(s). You can use flytekit.clis.sdk_in_container.register.register_all to achieve that.

In [9]:
from os import environ
from flytekit.configuration import set_flyte_config_file
from flytekit.contrib.notebook import python_notebook, spark_notebook
from flytekit.sdk.types import Types
from workflows.recipe_7.interactive_python import interactive_python

environ["FLYTE_INTERNAL_IMAGE"] = "docker.io/lyft/flytesnacks:datacouncil-{}".format(environ["version"])
set_flyte_config_file("production.config")
environ["FLYTE_INTERNAL_CONFIGURATION_PATH"] = "/root/production.config"

interactive_python.register(project="flytesnacks", domain="development", name="interactive_python", version=environ["version"])

'tsk:flytesnacks:development:interactive_python:123456789'

### 3) Launch a single task
This action runs the task on a flyte cluster (KinD, Docker for Desktop, EKS, GKE,... etc.)

In [10]:
from workflows.recipe_1.inner import IdentityWorkflow
from os import environ

exec = interactive_python.launch(project="flytesnacks", domain="development", inputs={
    'a': 5.0,
    'b': 6.0,
})
print_console_url(exec)

http://flyte.lyft.net/console/projects/flytesnacks/domains/development/executions/dw6jcj61xk


In [11]:
print("Waiting for workflow execution to complete")
exec.wait_for_completion()
print("Done!")
print()
print("Output: {}".format(exec.outputs["out"]))

Waiting for workflow execution to complete
Done!

Output: 11.0
